In [10]:
import cartopy.crs as ccrs
import hvplot.xarray
import xarray as xr
import requests
from glob import glob
import pandas as pd
from herbie import Herbie
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map
wave_wind_mapping = {
    # Combined Wave Parameters
    "Hsig": "HTSGW",       # Hsig: Significant wave height -> HTSGW: Significant Height of Combined Wind Waves and Swell
    "Dir": "DIRPW",      # PkDir: Peak wave direction -> DIRPW: Direction of Combined Wind Waves and Swell (Often peak direction is reported as the primary direction)
    "Tm01": "PERPW",       # Tm01: Mean wave period (m01) -> PERPW: Mean Period of Combined Wind Waves and Swell (Tm01 is a common representation for mean period)
    # Wind Parameters
    #"Windv_x": "UGRD",     # Windv_x: Horizontal component of wind velocity -> UGRD: U-component of wind
    #"Windv_y": "VGRD",     # Windv_y: Vertical component of wind velocity (Likely intended as the other horizontal component) -> VGRD: V-component of wind
}
files = pd.Series(sorted(glob("hcast_nc/*.nc")))
files = files[files.str.contains("(?:NI|SI)-(?:0|1).+-(?:HSIGN|DIR|RTM01)")].reset_index(drop=True)
files

0        hcast_nc/NI-000101_000630-DIR.nc
1      hcast_nc/NI-000101_000630-HSIGN.nc
2      hcast_nc/NI-000101_000630-RTM01.nc
3        hcast_nc/NI-000701_001231-DIR.nc
4      hcast_nc/NI-000701_001231-HSIGN.nc
                      ...                
193    hcast_nc/SI-180101_181231-HSIGN.nc
194    hcast_nc/SI-180101_181231-RTM01.nc
195      hcast_nc/SI-190101_191231-DIR.nc
196    hcast_nc/SI-190101_191231-HSIGN.nc
197    hcast_nc/SI-190101_191231-RTM01.nc
Length: 198, dtype: object

In [16]:
dsets = xr.open_mfdataset(reversed(files), combine="nested", compat="override")
dsets

<xarray.Dataset> Size: 39GB
Dimensions:    (time: 58440, latitude: 179, longitude: 154)
Coordinates:
  * time       (time) datetime64[ns] 468kB 2000-01-01 ... 2019-12-31T21:00:00
  * latitude   (latitude) float32 716B -48.04 -47.94 -47.85 ... -33.56 -33.46
  * longitude  (longitude) float32 616B 165.6 165.7 165.8 ... 179.4 179.5 179.6
Data variables:
    RTm01      (time, latitude, longitude) float64 13GB dask.array<chunksize=(2920, 103, 121), meta=np.ndarray>
    Hsig       (time, latitude, longitude) float64 13GB dask.array<chunksize=(2920, 103, 121), meta=np.ndarray>
    Dir        (time, latitude, longitude) float64 13GB dask.array<chunksize=(2920, 103, 121), meta=np.ndarray>

In [17]:
dsets.to_netcdf("Joao.nc", mode="w")